In [2]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Homework
The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.



# Q1. Downloading the data

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

- 16
- 17
- 18
- 19

In [ ]:
yellow_tr_data_01 = pd.read_parquet('/kaggle/input/yellow-tripdata-2023-01-02/yellow_tripdata_2023-01.parquet')
yellow_tr_data_02 = pd.read_parquet('/kaggle/input/yellow-tripdata-2023-01-02/yellow_tripdata_2023-02.parquet')
yellow_tr_data = pd.concat([yellow_tr_data_01, yellow_tr_data_02], ignore_index=True)

In [ ]:
yellow_tr_data_01.head()

In [ ]:
yellow_tr_data_01.dtypes

In [ ]:
yellow_tr_data_01.shape

# Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

- 32.59
- 42.59
- 52.59
- 62.59

In [ ]:
duration = (yellow_tr_data_01['tpep_dropoff_datetime'] - yellow_tr_data_01['tpep_pickup_datetime'])
yellow_tr_data_01['duration'] = duration.dt.total_seconds() / 60
duration_std = yellow_tr_data_01['duration'].std()
duration_std

In [ ]:
yellow_tr_data_01.head()

# Q3. Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

- 90%
- 92%
- 95%
- 98%

In [ ]:
filtered_data = yellow_tr_data_01.loc[(yellow_tr_data_01['duration'] > 1) & (yellow_tr_data_01['duration'] < 60)]
filtered_data.shape

In [ ]:
outliers_num = yellow_tr_data_01.shape[0] - filtered_data.shape[0]
records =  100 - ((outliers_num / yellow_tr_data_01.shape[0] ) * 100)
records

# Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
- Fit a dictionary vectorizer
- Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?
- 2
- 155
- 345
- 515
- 715


In [ ]:
categorical = ['PULocationID', 'DOLocationID']
filtered_data[categorical] = filtered_data[categorical].astype(str)

In [ ]:
filtered_data.shape

In [ ]:
data_dicts = filtered_data[categorical].to_dict(orient='records')
vectorizer = DictVectorizer()
feature_matrix = vectorizer.fit_transform(data_dicts)

In [ ]:
print("Shape of the Feature Matrix:", feature_matrix.shape)

# Q5. Training a model
Now let's use the feature matrix from the previous step to train a model. 

- Train a plain linear regression model with default parameters, where duration is the response variable
- Calculate the RMSE of the model on the training data

What's the RMSE on train?

- 3.64
- 7.64
- 11.64
- 16.64

In [ ]:
model = LinearRegression()
model.fit(feature_matrix, filtered_data['duration'])

In [ ]:
predictions = model.predict(feature_matrix)
rmse = mean_squared_error(filtered_data['duration'], predictions, squared=False)
print(f"RMSE: {rmse}")

# Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81

In [ ]:
duration_february = (yellow_tr_data_02['tpep_dropoff_datetime'] - yellow_tr_data_02['tpep_pickup_datetime'])
yellow_tr_data_02['duration'] = duration_february.dt.total_seconds() / 60
filtered_data = yellow_tr_data_02.loc[(yellow_tr_data_02['duration'] > 1) & (yellow_tr_data_02['duration'] < 60)]

In [ ]:
filtered_data[categorical] = filtered_data[categorical].astype(str)
data_dicts = filtered_data[categorical].to_dict(orient='records')
feature_matrix = vectorizer.transform(data_dicts)

In [ ]:
feature_matrix.shape

In [ ]:
predictions = model.predict(feature_matrix)

In [ ]:
rmse = mean_squared_error(filtered_data['duration'], predictions, squared=False)
print(f"RMSE: {rmse}")